### Model Testing
Chase Kent-Dotson  
April 4, 2022

### Introduction
This workbook is a collection of tests and training outputs that I did early on. I initially thought that saving the Keras model would allow me to access the history but that is not the case, so there are no evaluation graphs available for these attempts, only the text output. I also overwrote some of my early code so not all models are represented, but all moderately successful attempts from this notebook are scored at the top of the Final Model Training notebook, where my final model is built and trained completely. All models attempted are also included as complete files in the models directory, even those not represented in the code below.
  
This notebook likely will not run from top to bottom without some re-modification and the output will not be preserved if modified and rerun, but I wanted to include it to show the process I went through to begin understanding use of the data pipeline and model construction. It contains my most successful CNN which did not use pretrained weights, and an explanation of the optimization process I went through to improve it before using transfer learning models, then my early attempts at transfer learning.   

### Table of Contents:
* [CNN Modeling from Scratch](#2)
* [Early Transfer Learning Models](#3)
* [Conclusion](#4)

### Imports <a class="anchor" id="1"></a>

In [ ]:
# data handling
import numpy as np
import pandas as pd

# file directory interface
import os.path, sys

# neural network modeling
import tensorflow as tf 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D
from keras import Model

# transfer learning modeling
import efficientnet.tfkeras as efn

# data generation for modeling
from keras.preprocessing.image import ImageDataGenerator

### CNN Modeling from Scratch <a class="anchor" id="2"></a>
Below are my early attempts at modeling using CNNs from scratch without any pre-trained weights. 

In [2]:
# set variables for ease of optimization later
batch_size = 32
num_classes = 196
dropout = 0.2
img_size= (224,224)

In [3]:
#Data agumentation

# data generator with a 1/255 rescale, +-20% random shear and zoom, and a random binary flip
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   samplewise_std_normalization=True,
                                   samplewise_center=True,
                                   validation_split=0.2)

# no modifications to the test data
test_datagen = ImageDataGenerator(rescale=1./255,
                                  samplewise_std_normalization=True,
                                  samplewise_center=True)

# generate the data for the train set including automatic labels
train_data = train_datagen.flow_from_directory('data/car_data/car_data/train',
                                              target_size=img_size,
                                              shuffle=True,
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='training')

# generate the validation data from the portion set aside earlier
validation_data = train_datagen.flow_from_directory('data/car_data/car_data/train',
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='validation') 

# generate data for the test set
test_data = test_datagen.flow_from_directory('data/car_data/car_data/test',
                                              target_size=img_size,
                                              batch_size=batch_size,
                                              class_mode='categorical')

Found 6598 images belonging to 196 classes.
Found 1546 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [70]:
CNN_model = Sequential()

# Create simple CNN model architecture with Pooling for dimensionality reduction 
# and Dropout to reduce overfitting
CNN_model.add(Conv2D(32, kernel_size=(3, 3), activation = 'relu', input_shape = (128, 128, 3)))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(dropout))

# 2nd Convolution and Pooling Layer
CNN_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(dropout))

# 3rd Convolution and Pooling Layer
CNN_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(dropout))

# 4th Convolution and Pooling Layer
CNN_model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(dropout))

# 5th Convolution and Pooling Layer
CNN_model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
CNN_model.add(BatchNormalization())
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(dropout))

# Flatten the output of our convolutional layers
CNN_model.add(Flatten())

# Add dense layers
CNN_model.add(Dense(units=256,activation='relu'))
CNN_model.add(Dropout(dropout))
CNN_model.add(Dense(units=256,activation='relu'))
CNN_model.add(Dropout(dropout))
CNN_model.add(Dense(num_classes, activation='softmax'))

# Print out a summary of the network
CNN_model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_51 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_24 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_25 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 30, 30, 64)      

In [71]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with the desired loss function, optimizer, and metric(s) to track
CNN_model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer,
                  metrics = ['accuracy'])

In [72]:
history = CNN_model.fit(train_data,
                        steps_per_epoch=train_data.samples // batch_size,
                        validation_data=validation_data,
                        epochs=10,
                        validation_steps=validation_data.samples // batch_size)

Epoch 1/10
206/206 [==============================] - 241s 1s/step - loss: 5.3604 - accuracy: 0.0090 - val_loss: 5.3625 - val_accuracy: 0.0117
Epoch 2/10
206/206 [==============================] - 236s 1s/step - loss: 5.1967 - accuracy: 0.0128 - val_loss: 5.1474 - val_accuracy: 0.0169
Epoch 3/10
206/206 [==============================] - 234s 1s/step - loss: 5.0999 - accuracy: 0.0193 - val_loss: 5.0368 - val_accuracy: 0.0280
Epoch 4/10
206/206 [==============================] - 235s 1s/step - loss: 5.0186 - accuracy: 0.0225 - val_loss: 4.9464 - val_accuracy: 0.0332
Epoch 5/10
206/206 [==============================] - 244s 1s/step - loss: 4.9180 - accuracy: 0.0323 - val_loss: 4.9200 - val_accuracy: 0.0339
Epoch 6/10
206/206 [==============================] - 231s 1s/step - loss: 4.8197 - accuracy: 0.0353 - val_loss: 4.8212 - val_accuracy: 0.0319
Epoch 7/10
206/206 [==============================] - 234s 1s/step - loss: 4.7131 - accuracy: 0.0433 - val_loss: 4.6594 - val_accuracy: 0.0586

In [73]:
# Evaluate the model's performance on the test data
score = CNN_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 92s 364ms/step - loss: 4.1674 - accuracy: 0.0931
Test loss: 4.167438983917236
Test accuracy: 0.09314762055873871


In [74]:
CNN_model.save('models/model_2')

INFO:tensorflow:Assets written to: models/model_2\assets


In [75]:
reconstructed_model = keras.models.load_model('models/model_2')

In [76]:
r_history = reconstructed_model.fit(train_data,
                                    steps_per_epoch=train_data.samples // batch_size,
                                    validation_data=validation_data,
                                    epochs=30,
                                    validation_steps=validation_data.samples // batch_size)

Epoch 1/30
206/206 [==============================] - 241s 1s/step - loss: 4.2555 - accuracy: 0.0774 - val_loss: 4.1911 - val_accuracy: 0.0879
Epoch 2/30
206/206 [==============================] - 243s 1s/step - loss: 4.0933 - accuracy: 0.0903 - val_loss: 4.1650 - val_accuracy: 0.0885
Epoch 3/30
206/206 [==============================] - 227s 1s/step - loss: 3.9945 - accuracy: 0.1011 - val_loss: 4.2139 - val_accuracy: 0.0846
Epoch 4/30
206/206 [==============================] - 211s 1s/step - loss: 3.9030 - accuracy: 0.1087 - val_loss: 3.9838 - val_accuracy: 0.1094
Epoch 5/30
206/206 [==============================] - 211s 1s/step - loss: 3.7884 - accuracy: 0.1316 - val_loss: 3.9003 - val_accuracy: 0.1296
Epoch 6/30
206/206 [==============================] - 213s 1s/step - loss: 3.7034 - accuracy: 0.1377 - val_loss: 3.8055 - val_accuracy: 0.1335
Epoch 7/30
206/206 [==============================] - 244s 1s/step - loss: 3.6241 - accuracy: 0.1470 - val_loss: 3.7710 - val_accuracy: 0.1283

In [77]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 95s 378ms/step - loss: 2.8250 - accuracy: 0.3329
Test loss: 2.8250043392181396
Test accuracy: 0.33291879296302795


In [78]:
reconstructed_model.save('models/model_2')

INFO:tensorflow:Assets written to: models/model_2\assets


### CNN Modeling from Scratch: Findings
Above is the construction and training output for my most successful CNN without using pre-trained weights. The first CNN constructed did not use batch normalization, and did not contain the final 512 node convolutional layer. It's maximum accuracy was __7%__. The model above, model_2, surpassed that accuracy in the first 10 epochs, and had a final accuracy of __33%__ after 40 epochs of training when the validation loss minimization and accuracy begun to stop their progress. The addition of batch normalization was helpful in preventing overtaining as the trend of validation progress slowed much quicker in the first model without it and the validation scores remained closer to the training scores for longer.  
  
Given that this is a 196 class multi-classification problem, I gathered from my previous tests that in order to achieve an accuracy I'd be satisfied with out of a CNN from scratch I'd need to increase complextity and then massively increase measures to combat overtraining (like lowering the learning rate), while investing huge amounts of time for the model to learn the problem. I figured the best pathway forward was to see if using a base model with pre trained weights could help me achieve a respectable accuracy within the amount of time I had available, so I moved onto that next. 

### Early Transfer Learning Models <a class="anchor" id="3"></a>
Below are some of my first attempts at applying transfer learning models to the problem.

In [27]:
# set up the transfer learning model structure
base_model = efn.EfficientNetB0(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(train_data.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#  batch normalization will be trainable with the top layer
for layer in base_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

# compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, None, None, 3 864         input_6[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, None, None, 3 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, None, None, 3 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [28]:
history = model.fit_generator(generator=train_data,
                              steps_per_epoch=train_data.samples // batch_size + 1 ,
                              validation_data=validation_data,
                              validation_steps=validation_data.samples // batch_size + 1,
                              epochs=10,                           
                              workers=8,             
                              max_queue_size=32,             
                              verbose=1)

Epoch 1/10
207/207 [==============================] - 618s 3s/step - loss: 5.2677 - acc: 0.0121 - val_loss: 5.1613 - val_acc: 0.0194
Epoch 2/10
207/207 [==============================] - 668s 3s/step - loss: 5.0576 - acc: 0.0409 - val_loss: 5.0025 - val_acc: 0.0459
Epoch 3/10
207/207 [==============================] - 687s 3s/step - loss: 4.8754 - acc: 0.0838 - val_loss: 4.8390 - val_acc: 0.0886
Epoch 4/10
207/207 [==============================] - 605s 3s/step - loss: 4.6865 - acc: 0.1361 - val_loss: 4.6599 - val_acc: 0.1248
Epoch 5/10
207/207 [==============================] - 602s 3s/step - loss: 4.4894 - acc: 0.2028 - val_loss: 4.4864 - val_acc: 0.1578
Epoch 6/10
207/207 [==============================] - 815s 4s/step - loss: 4.2894 - acc: 0.2527 - val_loss: 4.2953 - val_acc: 0.2012
Epoch 7/10
207/207 [==============================] - 652s 3s/step - loss: 4.0920 - acc: 0.2978 - val_loss: 4.1313 - val_acc: 0.2219
Epoch 8/10
207/207 [==============================] - 592s 3s/step - 

In [29]:
# Evaluate the model's performance on the test data
score = model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 210s 832ms/step - loss: 3.6209 - acc: 0.3062
Test loss: 3.620945930480957
Test accuracy: 0.30618083477020264


In [30]:
model.save('models/transfer_model_B0_lr0001')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [9]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr0001')

In [10]:
r_history = reconstructed_model.fit_generator(generator=train_data,
                                              steps_per_epoch=train_data.samples // batch_size + 1 ,
                                              validation_data=validation_data,
                                              validation_steps=validation_data.samples // batch_size + 1,
                                              initial_epoch=10,
                                              epochs=20,                           
                                              workers=8,             
                                              max_queue_size=32,             
                                              verbose=1)

Epoch 11/20
207/207 [==============================] - 697s 3s/step - loss: 3.3180 - acc: 0.4400 - val_loss: 3.4755 - val_acc: 0.3299
Epoch 12/20
207/207 [==============================] - 678s 3s/step - loss: 3.1247 - acc: 0.4779 - val_loss: 3.3360 - val_acc: 0.3448
Epoch 13/20
207/207 [==============================] - 653s 3s/step - loss: 2.9571 - acc: 0.4950 - val_loss: 3.1979 - val_acc: 0.3687
Epoch 14/20
207/207 [==============================] - 642s 3s/step - loss: 2.8002 - acc: 0.5238 - val_loss: 3.0658 - val_acc: 0.3894
Epoch 15/20
207/207 [==============================] - 671s 3s/step - loss: 2.6442 - acc: 0.5446 - val_loss: 2.9321 - val_acc: 0.4172
Epoch 16/20
207/207 [==============================] - 670s 3s/step - loss: 2.5157 - acc: 0.5667 - val_loss: 2.8182 - val_acc: 0.4327
Epoch 17/20
207/207 [==============================] - 667s 3s/step - loss: 2.3719 - acc: 0.5861 - val_loss: 2.6952 - val_acc: 0.4470
Epoch 18/20
207/207 [==============================] - 684s 3s

In [11]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 202s 802ms/step - loss: 2.4187 - acc: 0.4957
Test loss: 2.4186792373657227
Test accuracy: 0.49570947885513306


In [13]:
reconstructed_model.save('models/transfer_model_B0_lr0001_20epoch')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001_20epoch\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [14]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr0001_20epoch')

In [15]:
r_history = reconstructed_model.fit_generator(generator=train_data,
                                              steps_per_epoch=train_data.samples // batch_size + 1 ,
                                              validation_data=validation_data,
                                              validation_steps=validation_data.samples // batch_size + 1,
                                              initial_epoch=20,
                                              epochs=30,                           
                                              workers=8,             
                                              max_queue_size=32,             
                                              verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 21/30
207/207 [==============================] - 586s 3s/step - loss: 1.9287 - acc: 0.6620 - val_loss: 2.3270 - val_acc: 0.5013
Epoch 22/30
207/207 [==============================] - 566s 3s/step - loss: 1.8050 - acc: 0.6844 - val_loss: 2.2610 - val_acc: 0.5201
Epoch 23/30
207/207 [==============================] - 561s 3s/step - loss: 1.7193 - acc: 0.6948 - val_loss: 2.1571 - val_acc: 0.5362
Epoch 24/30
207/207 [==============================] - 542s 3s/step - loss: 1.6331 - acc: 0.7178 - val_loss: 2.1034 - val_acc: 0.5310
Epoch 25/30
207/207 [==============================] - 566s 3s/step - loss: 1.5490 - acc: 0.7328 - val_loss: 2.0428 - val_acc: 0.5543
Epoch 26/30
207/207 [==============================] - 544s 3s/step - loss: 1.4879 - acc: 0.7373 - val_loss: 1.9793 - val_acc: 0.5692
Epoch 27/30
207/207 [==============================] - 518s 2s/step - loss: 1.4219 - acc: 0.7516 - val_loss: 1.9413 - val_acc: 0.5699
Epoch 28/30
207/207 [==============================] - 515s 2s

In [16]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 180s 714ms/step - loss: 1.7827 - acc: 0.5952
Test loss: 1.7826638221740723
Test accuracy: 0.5951995849609375


In [17]:
reconstructed_model.save('models/transfer_model_B0_lr0001_30epoch')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001_30epoch\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [18]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr0001_30epoch')

In [19]:
r_history = reconstructed_model.fit_generator(generator=train_data,
                                              steps_per_epoch=train_data.samples // batch_size + 1 ,
                                              validation_data=validation_data,
                                              validation_steps=validation_data.samples // batch_size + 1,
                                              initial_epoch=30,
                                              epochs=40,                           
                                              workers=8,             
                                              max_queue_size=32,             
                                              verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 31/40
207/207 [==============================] - 575s 3s/step - loss: 1.1983 - acc: 0.7822 - val_loss: 1.7368 - val_acc: 0.5957
Epoch 32/40
207/207 [==============================] - 553s 3s/step - loss: 1.1367 - acc: 0.7949 - val_loss: 1.6888 - val_acc: 0.6158
Epoch 33/40
207/207 [==============================] - 518s 2s/step - loss: 1.0864 - acc: 0.8039 - val_loss: 1.6513 - val_acc: 0.6177
Epoch 34/40
207/207 [==============================] - 545s 3s/step - loss: 1.0449 - acc: 0.8143 - val_loss: 1.6285 - val_acc: 0.6210
Epoch 35/40
207/207 [==============================] - 528s 3s/step - loss: 1.0026 - acc: 0.8192 - val_loss: 1.5796 - val_acc: 0.6352
Epoch 36/40
207/207 [==============================] - 525s 3s/step - loss: 0.9657 - acc: 0.8303 - val_loss: 1.5454 - val_acc: 0.6410
Epoch 37/40
207/207 [==============================] - 581s 3s/step - loss: 0.9136 - acc: 0.8415 - val_loss: 1.5107 - val_acc: 0.6397
Epoch 38/40
207/207 [==============================] - 567s 3s

In [20]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 259s 1s/step - loss: 1.4461 - acc: 0.6523
Test loss: 1.4460686445236206
Test accuracy: 0.6522820591926575


In [21]:
reconstructed_model.save('models/transfer_model_B0_lr0001_40epoch')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001_40epoch\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [4]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr0001_40epoch')

In [5]:
r_history = reconstructed_model.fit_generator(generator=train_data,
                                              steps_per_epoch=train_data.samples // batch_size + 1 ,
                                              validation_data=validation_data,
                                              validation_steps=validation_data.samples // batch_size + 1,
                                              initial_epoch=40,
                                              epochs=50,                           
                                              workers=8,             
                                              max_queue_size=32,             
                                              verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 41/50
207/207 [==============================] - 737s 3s/step - loss: 0.7994 - acc: 0.8589 - val_loss: 1.4139 - val_acc: 0.6591
Epoch 42/50
207/207 [==============================] - 741s 4s/step - loss: 0.7654 - acc: 0.8644 - val_loss: 1.3909 - val_acc: 0.6611
Epoch 43/50
207/207 [==============================] - 745s 4s/step - loss: 0.7257 - acc: 0.8715 - val_loss: 1.3735 - val_acc: 0.6656
Epoch 44/50
207/207 [==============================] - 719s 3s/step - loss: 0.7089 - acc: 0.8734 - val_loss: 1.3351 - val_acc: 0.6734
Epoch 45/50
207/207 [==============================] - 700s 3s/step - loss: 0.6790 - acc: 0.8842 - val_loss: 1.3400 - val_acc: 0.6617
Epoch 46/50
207/207 [==============================] - 669s 3s/step - loss: 0.6544 - acc: 0.8862 - val_loss: 1.3137 - val_acc: 0.6759
Epoch 47/50
207/207 [==============================] - 684s 3s/step - loss: 0.6373 - acc: 0.8875 - val_loss: 1.2872 - val_acc: 0.6811
Epoch 48/50
207/207 [==============================] - 672s 3s

In [6]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 238s 945ms/step - loss: 1.2616 - acc: 0.6862
Test loss: 1.2615721225738525
Test accuracy: 0.6862330436706543


In [7]:
reconstructed_model.save('models/transfer_model_B0_lr0001_50epoch')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001_50epoch\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [8]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr0001_50epoch')

In [9]:
r_history = reconstructed_model.fit_generator(generator=train_data,
                                              steps_per_epoch=train_data.samples // batch_size + 1,
                                              validation_data=validation_data,
                                              validation_steps=validation_data.samples // batch_size + 1,
                                              initial_epoch=50,
                                              epochs=60,                           
                                              workers=8,             
                                              max_queue_size=32,             
                                              verbose=1)

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 51/60
207/207 [==============================] - 699s 3s/step - loss: 0.5597 - acc: 0.9036 - val_loss: 1.2303 - val_acc: 0.6837
Epoch 52/60
207/207 [==============================] - 659s 3s/step - loss: 0.5339 - acc: 0.9083 - val_loss: 1.2156 - val_acc: 0.6979
Epoch 53/60
207/207 [==============================] - 616s 3s/step - loss: 0.5261 - acc: 0.9103 - val_loss: 1.2140 - val_acc: 0.6921
Epoch 54/60
207/207 [==============================] - 554s 3s/step - loss: 0.5026 - acc: 0.9135 - val_loss: 1.1968 - val_acc: 0.6876
Epoch 55/60
207/207 [==============================] - 551s 3s/step - loss: 0.4809 - acc: 0.9219 - val_loss: 1.1869 - val_acc: 0.7018
Epoch 56/60
207/207 [==============================] - 559s 3s/step - loss: 0.4706 - acc: 0.9219 - val_loss: 1.1880 - val_acc: 0.6863
Epoch 57/60
207/207 [==============================] - 537s 3s/step - loss: 0.4445 - acc: 0.9274 - val_loss: 1.1714 - val_acc: 0.6953
Epoch 58/60
207/207 [==============================] - 566s 3s

In [10]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 183s 724ms/step - loss: 1.1425 - acc: 0.7073
Test loss: 1.1425036191940308
Test accuracy: 0.7072503566741943


In [11]:
reconstructed_model.save('models/transfer_model_B0_lr0001_60epoch')

INFO:tensorflow:Assets written to: models/transfer_model_B0_lr0001_60epoch\assets


C:\ProgramData\Anaconda3\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [5]:
reconstructed_model = keras.models.load_model('models/transfer_model_B0_lr001')

In [6]:
# Evaluate the model's performance on the test data
score = reconstructed_model.evaluate(test_data, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

252/252 [==============================] - 235s 921ms/step - loss: 1.0135 - acc: 0.7346
Test loss: 1.0135035514831543
Test accuracy: 0.7346101403236389


### Early Transfer Learning Models: Findings
My early transfer learning models proved immediately more predictively powerful than the previous CNN attempts. I attempted models using different learning rates, and confirmed my research that lowering the learning rate helps to prevent overfitting. I attempted different varients of the EfficientNet base model. While this models were better than the original CNNs I attempted, the tuning required more research and thought which I address in my Final Model Training notebook while I make a push for deciding upon and training my final model in preparation for evaluation. 

### Conclusion <a class="anchor" id="4"></a>
Implementing a CNN model from scratch proved fruitful, considering a naiive model guessing the majority class every time would provide only __0.8%__ accuracy, however more predictive power is required to make the model worthwhile from a business perspective as marketing data will need to be reliable in order to provide value. Transfer learning has proven very helpful thus far in achieving this goal, but will require better tuning to maximize the model's capabilities. I will tackle final model decisions, building, training, and tuning in my next notebook.